In [1]:
import numpy as np
import matplotlib.pyplot as plt

def grad(x,function, h):
    n = len(x)
    fprime = np.zeros(n)
    for i in range(n):
        x_aux = np.copy(x)
        x_aux[i] = x[i]+h
        delante = function(x)
        x_aux[i] = x[i]-2*h
        atras = function(x_aux)
        fprime[i] = (delante-atras)/(2*h)
    return fprime

def hessian(x, function, h):
    n = len(x)
    hessian = np.zeros([n, n])
    x_aux = np.copy(x)
    
    for i in range(n):
        for j in range(n):
            x[i] = x[i]+h 
            x[j] = x[j]+h
            d = function(x)
            x[j] = x[j]-2*h
            a = function(x)
            #regresar al valor de x original
            x = np.copy(x_aux)

            x[i] = x[i]-h 
            x[j] = x[j]+h
            dd = function(x)
            x[j] = x[j]-2*h
            aa = function(x)
            #regresar al original
            x = np.copy(x_aux)
            
            #llenar matriz
            hessian[i][j] = (d-a-dd+aa)/(4*h*h)

    return hessian

def rosenbrock(x):
    n = len(x)
    suma = 0
    for i in range(n-1):
        suma += 100*(x[i+1]-x[i]**2)**2+(1-x[i])**2
    return suma

def wood(x):
    return sum((
        100*(x[0]*x[0] - x[1])**2,
        (x[0]-1)**2,
        (x[2]-1)**2,
        90*(x[2]*x[2] - x[3])**2,
        10.1*((x[1]-1)**2 + (x[3]-1)**2),
        19.8*(x[1]-1)*(x[3]-1),
        ))

def branin(x):
    a = 1.0
    b = 5.1 / (4*np.pi**2)
    c = 5.0 / (np.pi)
    r = 6.0
    s = 10.0
    t = 1.0 / (8*np.pi)
    
    return a*(x[1]-b*x[0]**2+c*x[0]-r)**2+s*(1-t)*np.cos(x[0])+s

In [2]:
def cauchy_newton_trust_region(x_k, funcion, delta, eta, max_delta, max_iter):
    for i in range(max_iter):
        f_k = funcion(x_k)
        g = rosen_der(x_k)
        norm_grad = np.linalg.norm(g)
        #print(norm_grad)
        B = rosen_hess(x_k)

        #calcular p_k
        semi_pos_cond = np.dot(np.dot(g.T, B), g)
        norm_semi_pos_cond = np.linalg.norm(semi_pos_cond)
        #cauchy
        if (norm_semi_pos_cond <= 0):
            tau = 1.0
        else: 
            tau = min(1, norm_grad**3/(delta*semi_pos_cond))
        pk = -1.0*(tau*delta/norm_grad)*g

        rho_num = funcion(x_k)-funcion(x_k+pk)
        rho_dev = -np.dot(g.T, pk)-(0.5)*np.dot(np.dot(pk.T, B), pk)
        rho = rho_num / rho_dev

        if (rho < 0.25):
            delta *= 0.25
        else:
            if (rho > 0.75 and np.linalg.norm(pk) == delta):
                delta = min(2*delta, max_delta)
        if (rho > eta):
            x_k += pk
    return x_k

## Rosenbrock function n = 100

In [4]:
'''Algoritmo 2):
Toma el paso de Newton si esta en la region de confianza, 
en otro caso, toma el paso de Cauchy'''
from scipy.optimize import rosen, rosen_der, rosen_hess
n = 100
x_k = np.ones(n) + np.random.uniform(-2, 2, n)
delta = 0.1
max_iter = 5000
eta = 0.2
max_delta = 1.0
res = np.ones(n)
mean = np.zeros(n)

for i in range(30):
    print(i)
    x_k = np.ones(n) + np.random.uniform(-2, 2, n)
    x = cauchy_newton_trust_region(x_k, rosen, delta, eta, max_delta, max_iter)
    mean[i] = (x[0]-res).mean()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [6]:
mean.mean()

-0.09966415958038675

## Funcion wood

In [7]:
def cauchy_newton_trust_region(x_k, h, funcion, delta, eta, max_delta, max_iter):
    for i in range(max_iter):
        f_k = funcion(x_k)
        g = grad(x_k, funcion, h)
        norm_grad = np.linalg.norm(g)
        #print(norm_grad)
        B = hessian(x_k, funcion, h)

        #calcular p_k
        semi_pos_cond = np.dot(np.dot(g.T, B), g)
        norm_semi_pos_cond = np.linalg.norm(semi_pos_cond)
        #cauchy
        if (norm_semi_pos_cond <= 0):
            tau = 1.0
        else: 
            tau = min(1, norm_grad**3/(delta*semi_pos_cond))
        pk = -1.0*(tau*delta/norm_grad)*g

        rho_num = funcion(x_k)-funcion(x_k+pk)
        rho_dev = -np.dot(g.T, pk)-(0.5)*np.dot(np.dot(pk.T, B), pk)
        rho = rho_num / rho_dev

        if (rho < 0.25):
            delta *= 0.25
        else:
            if (rho > 0.75 and np.linalg.norm(pk) == delta):
                delta = min(2*delta, max_delta)
        if (rho > eta):
            x_k += pk
    return x_k

In [12]:
n = 4
x_k = np.ones(n) + np.random.uniform(-2, 2, n)
max_iter = 1000
alpha = 0.02
delta = 2
eta = 0.1
tol = 0.01
h = 0.000001
res = np.ones(n)
mean = np.zeros(30)

for i in range(30):
    print(i)
    x_k = np.ones(n) + np.random.uniform(-2, 2, n)
    x = cauchy_newton_trust_region(x_k, h, wood, delta, eta, max_delta, max_iter)
    mean[i] = (x[0]-res).mean()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [14]:
mean.mean()

0.003911148317874173

## Funcion Branin

In [38]:
n = 2
x_k = np.zeros(n)
x_k[0] = np.pi + np.random.uniform(-2, 2)
x_k[1] = 2.275 + np.random.uniform(-2, 2)

max_iter = 10000
alpha = 0.02
delta = 2
eta = 0.1
tol = 0.01
h = 0.000001
res = np.zeros(n)
res[0] = np.pi
res[1] = 2.275

mean = np.zeros(30)

for i in range(30):
    print(i)
    x_k = np.zeros(n)
    x_k[0] = np.pi + np.random.uniform(-2, 2)
    x_k[1] = 2.275 + np.random.uniform(-2, 2)
    
    x = cauchy_newton_trust_region(x_k, h, branin, delta, eta, max_delta, max_iter)
    mean[i] = (x[0]-res).mean()

0


C:\Users\zaira\AppData\Local\Temp/ipykernel_8992/4151619687.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  tau = min(1, norm_grad**3/(delta*semi_pos_cond))
C:\Users\zaira\AppData\Local\Temp/ipykernel_8992/4151619687.py:21: RuntimeWarning: invalid value encountered in double_scalars
  rho = rho_num / rho_dev


1
2
3


C:\Users\zaira\AppData\Local\Temp/ipykernel_8992/4151619687.py:16: RuntimeWarning: overflow encountered in double_scalars
  tau = min(1, norm_grad**3/(delta*semi_pos_cond))
C:\Users\zaira\AppData\Local\Temp/ipykernel_8992/2396031115.py:69: RuntimeWarning: invalid value encountered in double_scalars
  return a*(x[1]-b*x[0]**2+c*x[0]-r)**2+s*(1-t)*np.cos(x[0])+s
C:\Users\zaira\AppData\Local\Temp/ipykernel_8992/2396031115.py:69: RuntimeWarning: invalid value encountered in cos
  return a*(x[1]-b*x[0]**2+c*x[0]-r)**2+s*(1-t)*np.cos(x[0])+s


4
5
6
7
8
9
10
11
12
13
14
15
16


C:\Users\zaira\AppData\Local\Temp/ipykernel_8992/4151619687.py:17: RuntimeWarning: divide by zero encountered in double_scalars
  pk = -1.0*(tau*delta/norm_grad)*g
C:\Users\zaira\AppData\Local\Temp/ipykernel_8992/4151619687.py:17: RuntimeWarning: invalid value encountered in multiply
  pk = -1.0*(tau*delta/norm_grad)*g


17
18
19
20
21
22
23
24
25
26
27
28
29


In [40]:
mean.mean()

0.6184705340106799